<a href="https://colab.research.google.com/github/Yazeedx0/GenAI/blob/main/Training-Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **First Train**

In [27]:
!pip install transformers
!pip install datasets
!pip install torch


In [ ]:
from datasets import load_dataset
import pandas as pd
df = pd.read_csv('/content/Fully dataset.csv')


dataset = load_dataset('csv', data_files={'train': '/content/Fully dataset.csv'})

print(dataset['train'][0])
df['text'] = df['Text']
df.head()


In [29]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

# تحميل الموديل والـ Tokenizer
model = GPT2LMHeadModel.from_pretrained("/content/model_files")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/model_files")

tokenizer.pad_token = tokenizer.eos_token

# تجهيز البيانات
encodings = tokenizer(list(df['text']), truncation=True, padding=True, max_length=512, return_tensors='pt')

if 'token_type_ids' in encodings:
    encodings.pop('token_type_ids')

class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.input_ids = encodings['input_ids']
        self.attention_mask = encodings['attention_mask']

    def __len__(self):
        return self.input_ids.size(0)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.input_ids[idx].clone()
        }
        return item

dataset = CustomDataset(encodings)

# إعدادات التدريب
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)
trainer.train()


Step,Training Loss
10,0.671400
20,0.487000
30,0.653900
40,0.714900
50,0.655200
60,0.610900
70,0.479900
80,0.408000
90,0.543500
100,0.476300


TrainOutput(global_step=3876, training_loss=0.33474420486219897, metrics={'train_runtime': 1193.6815, 'train_samples_per_second': 12.986, 'train_steps_per_second': 3.247, 'total_flos': 1202429187072000.0, 'train_loss': 0.33474420486219897, 'epoch': 3.0})

In [36]:

model.save_pretrained("./arabian_gpt_trained")
tokenizer.save_pretrained("./arabian_gpt_trained")

('./arabian_gpt_trained/tokenizer_config.json',
 './arabian_gpt_trained/special_tokens_map.json',
 './arabian_gpt_trained/tokenizer.json')

In [ ]:
!pip install gradio

In [37]:
import gradio as gr
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch

model = GPT2LMHeadModel.from_pretrained("/content/trained_model-1")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/trained_model-1")

tokenizer.pad_token = tokenizer.eos_token

def generate_text(prompt, temperature=1.0):
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=150,
            temperature=temperature,
            top_k=50,
            top_p=0.95,
            do_sample=True
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if generated_text.lower().startswith(prompt.lower()):
        generated_text = generated_text[len(prompt):].strip()

    return generated_text

interface = gr.Interface(
    fn=generate_text,
    inputs=[gr.Textbox(label="Prompt"), gr.Slider(0, 2, step=0.1, value=1.0, label="Temperature")],
    outputs="text",
    title="اهلا بك في مساعد للجامعة الهاشمية تفضل!",
    description="أدخل نصًا للبدء بتوليد نصوص مشابهة بناءً على النموذج المدرب."
)

interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4406c8d20939dd1426.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [38]:
import shutil

source_path = '/content/trained_model-1'

destination_path = '/content/drive/My Drive/trained_model-1'

shutil.copytree(source_path, destination_path)


'/content/drive/My Drive/trained_model-1'

# **Second Train**

In [ ]:
from datasets import load_dataset
import pandas as pd
df = pd.read_csv('/content/label text.csv')


dataset = load_dataset('csv', data_files={'train': '/content/label text.csv'})

print(dataset['train'][0])
df['text'] = df['label\ttext']
df.head()


In [ ]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

model = GPT2LMHeadModel.from_pretrained("/content/arabian_gpt_trained")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/arabian_gpt_trained")

tokenizer.pad_token = tokenizer.eos_token

encodings = tokenizer(list(df['text']), truncation=True, padding=True, max_length=512, return_tensors='pt')

if 'token_type_ids' in encodings:
    encodings.pop('token_type_ids')

class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.input_ids = encodings['input_ids']
        self.attention_mask = encodings['attention_mask']

    def __len__(self):
        return self.input_ids.size(0)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.input_ids[idx].clone()
        }
        return item

dataset = CustomDataset(encodings)

training_args = TrainingArguments(
    output_dir='./results-5',
    eval_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)
trainer.train()


Step,Training Loss
10,1.444300
20,0.798000
30,0.727000
40,0.942900
50,0.651800
60,0.732300
70,0.938000
80,0.862900
90,0.957400
100,0.840200


In [ ]:
model.save_pretrained("./arabian_gpt_trained-2")
tokenizer.save_pretrained("./arabian_gpt_trained-2")
import shutil

source_path = './arabian_gpt_trained-2'
destination_path = '/content/drive/My Drive/arabian_gpt_trained-2'
shutil.copytree(source_path, destination_path)

print("تم نقل الموديل إلى Google Drive بنجاح.")
